## 1. Import libraries

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

## 2. Load the Iris Dataset

In [2]:
iris_data = load_iris()
X = iris_data["data"]
y = iris_data["target"]

In [3]:
X.dtype, y.dtype

(dtype('float64'), dtype('int32'))

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=17)

In [5]:
X_train, y_train = torch.Tensor(X_train), torch.LongTensor(y_train)
X_test, y_test = torch.Tensor(X_test), torch.LongTensor(y_test)


## 3. Build a Model

In [6]:
class IrisClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4, 16)
        self.fc2 = nn.Linear(16, 3)
    
    def forward(self, X):
        return self.fc2(F.relu(self.fc1(X)))


In [7]:
model = IrisClassifier()

## 4. Setting up Hyperparameters

In [8]:
lr = 1e-3
epochs = 10_000
device = "cuda" if torch.cuda.is_available() else "cpu"
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

## 5. Moving to Device

In [9]:
X_train = X_train.to(device)
X_test = X_test.to(device)
y_train = y_train.to(device)
y_test = y_test.to(device)
model = model.to(device)

## 6. Training the Model

In [10]:
for epoch in range(epochs):
    model.train()
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 1000 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [1000/10000], Loss: 0.1121
Epoch [2000/10000], Loss: 0.0557
Epoch [3000/10000], Loss: 0.0428
Epoch [4000/10000], Loss: 0.0357
Epoch [5000/10000], Loss: 0.0306
Epoch [6000/10000], Loss: 0.0266
Epoch [7000/10000], Loss: 0.0234
Epoch [8000/10000], Loss: 0.0211
Epoch [9000/10000], Loss: 0.0195
Epoch [10000/10000], Loss: 0.0185


## 7. Predictions

In [11]:
model.eval()
with torch.inference_mode():
    y_pred = model(X_test)
    y_pred = torch.argmax(y_pred, dim=1)
    accuracy = (y_pred == y_test).sum().item() / y_test.size(dim=0)
    print(f"Test Accuracy: {accuracy:.2f}")


Test Accuracy: 0.97
